In [1]:
# !pip install twint

In [2]:
import twint

ModuleNotFoundError: No module named 'twint'

In [3]:
import folium
import h3
from shapely.geometry import Polygon, shape
import pandas as pd
import shapely
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()
import twint
import os
import time

ModuleNotFoundError: No module named 'twint'

In [ ]:
# !pip install folium
# !pip install h3
# pip install shapely


In [ ]:
import folium
import h3
from shapely.geometry import Polygon, shape
import pandas as pd
import shapely
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()
import twint
import os
import time

In [4]:
# A GeoJSON object representing a bounding box for the region of interest

geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
             -6.121381,
            49.316506


            ],
            [
              -5.329848,
                55.070800

            ],
            [
              -8.209109,
                57.498302

            ],
            [
              -2.404535,
                59.606828,

            ],
            [
             1.948987,
                 51.364921,

            ],
            [
               -6.121381,
            49.316506
            
            ]
          ]
        ]
      }
    }
  ]
}

In [5]:
# Add a 1 degree buffer around this area

s = shape(geojson['features'][0]['geometry'])
s = s.buffer(1)
feature = {'type': 'Feature', 'properties': {}, 'geometry': shapely.geometry.mapping(s)}
feature['geometry']['coordinates'] = [[[v[0], v[1]] for v in feature['geometry']['coordinates'][0]]]
feature = feature['geometry']
feature['coordinates'][0] = [[v[1], v[0]] for v in feature['coordinates'][0]]

In [6]:
# map H3 hexagons (code taken from H3 example: https://github.com/uber/h3-py-notebooks/blob/master/notebooks/usage.ipynb)

def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=14, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=14, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

# find all hexagons with a center that falls within our buffered area of interest from above
polyline = feature['coordinates'][0]
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
m.add_child(my_PolyLine)

# make the list of hexagon IDs in our AOI 
hexagons = list(h3.polyfill(feature, 5))

# map the hexagons
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v:v[0],polyline))
    lng.extend(map(lambda v:v[1],polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color='red')
    m.add_child(my_PolyLine)
display(m)

In [7]:
len(hexagons)


2932

In [8]:
np.sqrt((h3.cell_area(hexagons[0], unit='km^2')/0.827)/np.pi)


9.224958034031982

In [9]:
for hex in hexagons:
    center = h3.h3_to_geo(hex)
    r = np.sqrt((h3.cell_area(hex, unit='km^2')/0.827)/np.pi) 
    query = "(nandos OR chicken) AND geocode:" + str(center[0]) + "," + str(center[1]) + "," + str(r) + "km"
    
print("Example query:", query)

Example query: (nandos OR chicken) AND geocode:57.69093436243683,-2.360469554544234,8.97659666526597km


In [10]:
os.mkdir('data_by_hex1')


NameError: name 'os' is not defined

In [13]:
# Download tweets using TWINT for each hexagon in the list
hexagons_left = hexagons

while len(hexagons_left) > 0:
    hex = hexagons_left.pop(0)
    
    if not os.path.exists('data_by_hex1/' + hex + '.csv'):
        center = h3.h3_to_geo(hex)
        r = np.sqrt(h3.cell_area(hex, unit='km^2')*1.20919957616/np.pi) 
        query = "(nandos OR chicken) AND geocode:" + str(center[0]) + "," + str(center[1]) + "," + str(r) + "km"
        print(query)


        c = twint.Config()

        c.Search = query
        c.Store_csv = True
        c.Output = "data_by_hex1/" + hex + ".csv"
        c.Hide_output = True
        c.Since = "2021-01-01"

        try:
            twint.run.Search(c)
        except:
            os.remove('data_by_hex1/' + hex + '.csv')
            time.sleep(10)
            print('error, removing this one')
            hexagons_left.append(hex)
    
    print(len(hexagons_left), "remaining")

CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)


(nandos OR chicken) AND geocode:58.37047065111409,-6.251264615322001,9.07918076417903km


sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 64.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 125.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDataExpecting value: line 1 column 1 (char 0)
sleeping for 216.0 secs


FileNotFoundError: [Errno 2] No such file or directory: 'data_by_hex1/851901affffffff.csv'

In [ ]:
pip uninstall twint --y

In [ ]:
pip install --user --upgrade "git+https://github.com/twintproject/twint.git@origin/master#egg=twint"